# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS109A Introduction to Data Science 

## Lab 2: Pandas and Web Scraping with Beautiful Soup

**Harvard University**<br>
**Fall 2019**<br>
**Instructors:** Pavlos Protopapas, Kevin Rader, and Chris Tanner<br>
**Lab Instructors:** Chris Tanner and Eleni Kaxiras <br>
**Authors:** Rahul Dave, David Sondak, Will Claybaugh, Pavlos Protopapas, Chris Tanner, Eleni Kaxiras

---

In [3]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
from IPython.core.display import HTML
def css_styling():
    styles = open("/Users/emma/Documents/Data_Science/Harvard_course/2019-CS109A/content/styles/cs109.css", "r").read()
    return HTML(styles)
css_styling()

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
</ol>

## Learning Goals

About 6,000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  


By the end of this lab, you should be able to:

- Load and systematically address missing values, ancoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.

*This lab corresponds to lectures #1, #2, and #3 and maps on to homework #1 and further.*

### Basic EDA workflow

(From the lecture, repeated here for convenience).

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas 
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning. 

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [113]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [114]:
#Read the data into a dataframe
df = pd.read_csv("/Users/emma/Documents/Data_Science/Harvard_course/2019-CS109A/content/labs/lab02/data/goodreads.csv", encoding='utf-8')

#Examine the first few rows of the dataframe
df

,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.Suzanne_Collins,2008,/genres/young-adult|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/book-club|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
0,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
1,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
2,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
3,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice
4,4.25,12363,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936.0,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470,Gone with the Wind
...,...,...,...,...,...,...,...,...,...,...
5994,4.17,2226,0767913736,good_reads:book,https://www.goodreads.com/author/show/44565.Ca...,2005.0,/genres/history|/genres/non-fiction|/genres/bi...,dir60/78508.The_River_of_Doubt.html,16618,The River of Doubt
5995,3.99,775,1416909427,good_reads:book,https://www.goodreads.com/author/show/151371.J...,2006.0,/genres/young-adult|/genres/realistic-fiction|...,dir60/259068.Shug.html,6179,Shug
5996,3.78,540,1620612321,good_reads:book,https://www.goodreads.com/author/show/5761314....,2012.0,/genres/contemporary|/genres/romance|/genres/y...,dir60/13503247-flawed.html,2971,Flawed
5997,3.91,281,NaN,good_reads:book,https://www.goodreads.com/author/show/1201952....,2006.0,/genres/religion|/genres/islam|/genres/religio...,dir60/2750008.html,3083,Ø£Ø³Ø¹Ø¯ Ø§Ù Ø±Ø£Ø© ÙÙ Ø§ÙØ¹Ø§ÙÙ


Oh dear. That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`

<div class="exercise"><b>Exercise</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)


In [115]:
# your code here
df=pd.read_csv("/Users/emma/Documents/Data_Science/Harvard_course/2019-CS109A/content/labs/lab02/data/goodreads.csv", header=None,
               names=["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name'],
)

#Examine the first few rows of the dataframe
df.head()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
0,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008.0,/genres/young-adult|/genres/science-fiction|/g...,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
1,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
2,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
3,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
4,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice


### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content. 

<div class="exercise"><b>Exercise</b></div>
Lets check the types of the columns. What do you find?

In [9]:
# your code here
####### 
df.dtypes
####### 


rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

*your answer here*

Notice that `review_count` and `rating_counts` are objects instead of ints, and the `year` is a float!

There are a couple more quick sanity checks to perform on the dataframe. 

In [10]:
print(df.shape)
df.columns

(6000, 10)


Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'genre_urls', 'dir', 'rating_count', 'name'], dtype='object')

### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.

In [116]:
#Get a sense of how many missing values there are in the dataframe.
print(np.sum([df.rating.isnull()]))
print(np.sum([df.review_count.isnull()]))
print(np.sum([df.isbn.isnull()]))
print(np.sum([df.booktype.isnull()]))
print(np.sum([df.author_url.isnull()]))
print(np.sum([df.year.isnull()]))
print(np.sum([df.genre_urls.isnull()]))
print(np.sum([df.dir.isnull()]))
print(np.sum([df.rating_count.isnull()]))
print(np.sum([df.name.isnull()]))

2
0
475
0
0
7
62
0
0
0


In [12]:
#Try to locate where the missing values occur
df[df.rating.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
3643,NaN,None,None,None,None,NaN,NaN,dir37/9658936-harry-potter.html,None,None
5282,NaN,None,None,None,None,NaN,NaN,dir53/113138.The_Winner.html,None,None


How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

In [17]:
np.mean(df.rating)

4.042200733577858

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [23]:
df[df.year.isnull()].shape
df[df.year.isnull()]

#df.year.isnull()


,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
2442,4.23,526,NaN,good_reads:book,https://www.goodreads.com/author/show/623606.A...,NaN,/genres/religion|/genres/islam|/genres/non-fic...,dir25/1301625.La_Tahzan.html,4134,La Tahzan
2869,4.61,2,NaN,good_reads:book,https://www.goodreads.com/author/show/8182217....,NaN,NaN,dir29/22031070-my-death-experiences---a-preach...,23,My Death Experiences - A Preacherâs 18 Apoca...
3643,NaN,None,None,None,None,NaN,NaN,dir37/9658936-harry-potter.html,None,None
5282,NaN,None,None,None,None,NaN,NaN,dir53/113138.The_Winner.html,None,None
5572,3.71,35,8423336603,good_reads:book,https://www.goodreads.com/author/show/285658.E...,NaN,/genres/fiction,dir56/890680._rase_una_vez_el_amor_pero_tuve_q...,403,Ãrase una vez el amor pero tuve que matarlo. ...
5658,4.32,44,NaN,good_reads:book,https://www.goodreads.com/author/show/25307.Ro...,NaN,/genres/fantasy|/genres/fantasy|/genres/epic-f...,dir57/5533041-assassin-s-apprentice-royal-assa...,3850,Assassin's Apprentice / Royal Assassin (Farsee...
5683,4.56,204,NaN,good_reads:book,https://www.goodreads.com/author/show/3097905....,NaN,/genres/fantasy|/genres/young-adult|/genres/ro...,dir57/12474623-tiger-s-dream.html,895,"Tiger's Dream (The Tiger Saga, #5)"


### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values? 

In [117]:
#Treat the missing or invalid values in your dataframe
####### 
df[df.year.notnull()].shape
df = df[df.year.notnull()]

Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [27]:
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

In [12]:
print(np.sum(df.year.isnull()))
df.shape # We removed seven rows

0


(5993, 10)

<div class="exercise"><b>Exercise</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.

In [118]:
df.year = df.year.astype(int)
df.rating_count=df.rating_count.astype(int)
df.review_count = df.review_count.astype(int)

In [13]:
# your code here
df.rating_count=df.rating_count.astype(int)
df.review_count=df.review_count.astype(int)
df.year=df.year.astype(int)

Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [31]:
df.dtypes

rating          float64
review_count      int64
isbn             object
booktype         object
author_url       object
year              int64
genre_urls       object
dir              object
rating_count      int64
name             object
dtype: object

Sweet!

Some of the other colums that should be strings have NaN.

In [40]:
df.loc[df.genre_urls.isnull()].shape
df.loc[df.isbn.isnull()].shape

(0, 10)

In [119]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

##  Part 2: Parsing and Completing the Data Frame 

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

In [43]:
df.author_url[1]
df.genre_urls[1]

'/genres/fantasy|/genres/young-adult|/genres/fiction|/genres/fantasy|/genres/magic|/genres/childrens|/genres/adventure|/genres/science-fiction-fantasy'

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [44]:
#Get the first author_url
test_string = df.author_url[0]
test_string

'https://www.goodreads.com/author/show/153394.Suzanne_Collins'

In [50]:
test_string.split('/')[-1].split('.')[-1]

'Suzanne_Collins'

In [17]:
#Test out some string operations to isolate the author name

test_string.split('/')[-1].split('.')[1:][0]

'Suzanne_Collins'

<div class="exercise"><b>Exercise</b></div>

Lets wrap the above code into a function which we will then use

In [120]:
def get_author(url):
    return url.split('/')[-1].split('.')[-1]

df['author']=df.author_url.map(get_author)


In [18]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above
def get_author(url):
    # your code here
    name = url.split('/')[-1].split('.')[1:][0]
    ####### 
    return name

In [19]:
#Apply the get_author function to the 'author_url' column using '.map' 
#and add a new column 'author' to store the names
df['author'] = df.author_url.map(get_author)
df.author[0:5]

0    Suzanne_Collins
1        J_K_Rowling
2    Stephenie_Meyer
3         Harper_Lee
4        Jane_Austen
Name: author, dtype: object

<div class="exercise"><b>Exercise</b></div>

Now parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [76]:
test_string=df.genre_urls[0]

In [96]:
genre_list=df.genre_urls[0].split('|')
all_genres = []
for i in genre_list:
    genre_temp = i.split('/')[-1]
    all_genres.append(genre_temp)
    "|".join(all_genres)


In [95]:
genre_split = df.genre_urls[0].split('|')
all_genres=[i.split('/')[-1] for i in genre_split]
genres="|".join(all_genres)
genres

'young-adult|science-fiction|dystopia|fantasy|science-fiction|romance|adventure|book-club|young-adult|teen|apocalyptic|post-apocalyptic|action'

Define function to turn a genre url into a list of genres

In [121]:
def id_genres(url):
    genre_split=url.split('|')
    all_genres=[i.split('/')[-1] for i in genre_split]
    return "|".join(all_genres)


Test function with first genre_url

In [101]:
test_string = df.genre_urls[0]
id_genres(test_string)

'young-adult|science-fiction|dystopia|fantasy|science-fiction|romance|adventure|book-club|young-adult|teen|apocalyptic|post-apocalyptic|action'

Create new column in dataframe for genre list

In [122]:
df['genres']= df.genre_urls.map(id_genres)

In [123]:

df.genre_urls.head()

0    /genres/young-adult|/genres/science-fiction|/g...
1    /genres/fantasy|/genres/young-adult|/genres/fi...
2    /genres/young-adult|/genres/fantasy|/genres/ro...
3    /genres/classics|/genres/fiction|/genres/histo...
4    /genres/classics|/genres/fiction|/genres/roman...
Name: genre_urls, dtype: object

In [90]:
df.genre_urls[0].strip().split('|')


['/genres/young-adult',
 '/genres/science-fiction',
 '/genres/dystopia',
 '/genres/fantasy',
 '/genres/science-fiction',
 '/genres/romance',
 '/genres/adventure',
 '/genres/book-club',
 '/genres/young-adult',
 '/genres/teen',
 '/genres/apocalyptic',
 '/genres/post-apocalyptic',
 '/genres/action']

In [21]:
# your code here
#Examine some examples of genre_urls

#Test out some string operations to isolate the genre name
test_genre_string=df.genre_urls[0]
genres=test_genre_string.strip().split('|')
for e in genres:
    print(e.split('/')[-1])
    "|".join(genres)

young-adult
science-fiction
dystopia
fantasy
science-fiction
romance
adventure
book-club
young-adult
teen
apocalyptic
post-apocalyptic
action


<div class="exercise"><b>Exercise</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [22]:
def split_and_join_genres(url):
    # your code here
    genres=url.strip().split('|')
    genres=[e.split('/')[-1] for e in genres]
    return "|".join(genres)

Test your function

In [23]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

'young-adult|science-fiction'

In [24]:
split_and_join_genres("")

''

<div class="exercise"><b>Exercise</b></div>

Use map again to create a new "genres" column

In [25]:

df['genres']=df.genre_urls.map(split_and_join_genres)
df.head()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
0,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008,/genres/young-adult|/genres/science-fiction|/g...,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)",Suzanne_Collins,young-adult|science-fiction|dystopia|fantasy|s...
1,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...,J_K_Rowling,fantasy|young-adult|fiction|fantasy|magic|chil...
2,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)",Stephenie_Meyer,young-adult|fantasy|romance|paranormal|vampire...
3,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird,Harper_Lee,classics|fiction|historical-fiction|academic|s...
4,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice,Jane_Austen,classics|fiction|romance|historical-fiction|li...


Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [124]:
df[df.author == "Marguerite_Yourcenar"]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
1014,4.23,483,0374529264,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1951,/genres/historical-fiction|/genres/fiction|/ge...,dir11/12172.Memoirs_of_Hadrian.html,6258,Memoirs of Hadrian,Marguerite_Yourcenar,historical-fiction|fiction|cultural|france|cla...
5620,4.11,74,2070367983,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1968,/genres/fiction|/genres/historical-fiction|/ge...,dir57/953435.L_uvre_au_noir.html,1601,L'Åuvre au noir,Marguerite_Yourcenar,fiction|historical-fiction|cultural|france|eur...


Let us delete the `genre_urls` column.

In [125]:
del df['genre_urls']
df.head()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,author,genres
0,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)",Suzanne_Collins,young-adult|science-fiction|dystopia|fantasy|s...
1,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...,J_K_Rowling,fantasy|young-adult|fiction|fantasy|magic|chil...
2,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)",Stephenie_Meyer,young-adult|fantasy|romance|paranormal|vampire...
3,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird,Harper_Lee,classics|fiction|historical-fiction|academic|s...
4,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice,Jane_Austen,classics|fiction|romance|historical-fiction|li...


And then save the dataframe out!

In [127]:
df.to_csv("/Users/emma/Documents/Data_Science/Harvard_course/2019-CS109A/content/labs/lab02/data/cleaned-goodreads.csv", index=False, header=True)

---

## Part 3: Grouping 

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [129]:
df[df.year<0]

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,author,genres
47,3.68,5785,0143039954,good_reads:book,https://www.goodreads.com/author/show/903.Homer,-800,dir01/1381.The_Odyssey.html,560248,The Odyssey,Homer,classics|fiction|poetry|fantasy|mythology|acad...
246,4.01,365,0147712556,good_reads:book,https://www.goodreads.com/author/show/903.Homer,-800,dir03/1375.The_Iliad_The_Odyssey.html,35123,The Iliad/The Odyssey,Homer,classics|fantasy|mythology|fantasy|academic|sc...
455,3.85,1499,0140449140,good_reads:book,https://www.goodreads.com/author/show/879.Plato,-380,dir05/30289.The_Republic.html,82022,The Republic,Plato,philosophy|classics|non-fiction|politics|histo...
596,3.77,1240,0679729526,good_reads:book,https://www.goodreads.com/author/show/919.Virgil,-29,dir06/12914.The_Aeneid.html,60308,The Aeneid,Virgil,classics|poetry|fiction|fantasy|mythology|acad...
629,3.64,1231,1580495931,good_reads:book,https://www.goodreads.com/author/show/1002.Sop...,-429,dir07/1554.Oedipus_Rex.html,93192,Oedipus Rex,Sophocles,classics|plays|drama|fiction|academic|school|l...
674,3.92,3559,1590302257,good_reads:book,https://www.goodreads.com/author/show/1771.Sun...,-512,dir07/10534.The_Art_of_War.html,114619,The Art of War,Sun_Tzu,non-fiction|politics|classics|literature|psych...
746,4.06,1087,0140449183,good_reads:book,https://www.goodreads.com/author/show/5158478....,-500,dir08/99944.The_Bhagavad_Gita.html,31634,The Bhagavad Gita,Anonymous,classics|spirituality|religion|hinduism|fantas...
777,3.52,1038,1580493882,good_reads:book,https://www.goodreads.com/author/show/1002.Sop...,-442,dir08/7728.Antigone.html,49084,Antigone,Sophocles,drama|fiction|classics|academic|read-for-schoo...
1233,3.94,704,015602764X,good_reads:book,https://www.goodreads.com/author/show/1002.Sop...,-400,dir13/1540.The_Oedipus_Cycle.html,36008,The Oedipus Cycle,Sophocles,classics|plays|drama|fiction|literature|academ...
1397,4.03,890,0192840509,good_reads:book,https://www.goodreads.com/author/show/12452.Aesop,-560,dir14/21348.Aesop_s_Fables.html,71259,Aesop's Fables,Aesop,classics|childrens|literature|fantasy|fairy-ta...


In [30]:
# your code here
df[df.year < 0].name
#These are books written before the Common Era (BCE, equivalent to BC).

47                               The Odyssey
246                    The Iliad/The Odyssey
455                             The Republic
596                               The Aeneid
629                              Oedipus Rex
674                           The Art of War
746                        The Bhagavad Gita
777                                 Antigone
1233                       The Oedipus Cycle
1397                          Aesop's Fables
1398                   The Epic of Gilgamesh
1428                                   Medea
1815                            The Oresteia
1882         The Trial and Death of Socrates
2078    The History of the Peloponnesian War
2527                           The Histories
3133                          Complete Works
3274                  The Nicomachean Ethics
3757                              Lysistrata
4402                           The Symposium
4475                                 Apology
5367                          Five Dialogues
Name: name

We can determine the "best book" by year! For this we use Panda's `groupby()`. `Groupby()` allows grouping a dataframe by any (usually categorical) variable. Would it make sense to ever groupby integer variables? Floating point variables?

In [135]:
dfgb_author = df.groupby('author')
numbooks_per_author = dfgb_author['author'].count()
numbooks_per_author

author
A_A_Milne                 6
A_G_Howard                1
A_J_Cronin                1
A_J_Jacobs                1
A_J_Salt                  1
                         ..
_                        42
_gota_Krist_f             1
_mile_Zola                4
_ric_Emmanuel_Schmitt     1
_sne_Seierstad            1
Name: author, Length: 2645, dtype: int64

In [31]:
dfgb_author = df.groupby('author')
type(dfgb_author)

pandas.core.groupby.generic.DataFrameGroupBy

Perhaps we want the number of books each author wrote

In [32]:
dfgb_author.count()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,genres
author,,,,,,,,,,
A_A_Milne,6,6,6,6,6,6,6,6,6,6
A_G_Howard,1,1,1,1,1,1,1,1,1,1
A_J_Cronin,1,1,1,1,1,1,1,1,1,1
A_J_Jacobs,1,1,1,1,1,1,1,1,1,1
A_J_Salt,1,1,1,1,1,1,1,1,1,1
A_Meredith_Walters,2,2,2,2,2,2,2,2,2,2
A_N_Roquelaure,2,2,2,2,2,2,2,2,2,2
A_S_Byatt,1,1,1,1,1,1,1,1,1,1
A_S_King,1,1,1,1,1,1,1,1,1,1


Lots of useless info there. One column should suffice

### Exercise:

- Group the dataframe by `author`. Include the following columns: `rating`, `name`, `author`. For the aggregation of the `name` column which includes the names of the books create a list with the strings containing the name of each book. Make sure that the way you aggregate the rest of the columns make sense! 

- Create a new column with number of books for each author and find the most prolific author!

In [141]:
# mean ratings for each author
mean_ratings = dfgb_author.rating.mean()

# list of books by each author
list_of_books = dfgb_author.name
list_of_books[0]

IndexError: Column(s) name already selected

In [142]:
###### Before we start : what do we do about these titles where 'name' is unreadable? Try different encodings?
auth_name = 'A_id_al_Qarni'
df[df.author == auth_name].head()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,author,genres
2213,4.19,1169,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2003,dir23/2750180.html,15781,ÙØ§ ØªØ­Ø²Ù,A_id_al_Qarni,religion|religion|islam|self-help|non-fiction|...
5998,3.91,281,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2006,dir60/2750008.html,3083,Ø£Ø³Ø¹Ø¯ Ø§Ù Ø±Ø£Ø© ÙÙ Ø§ÙØ¹Ø§ÙÙ,A_id_al_Qarni,religion|islam|religion|self-help|spirituality...


In [143]:
df[df.author == auth_name].iat[0,8].encode('UTF-16')

b'\xff\xfe\xd9\x00\x84\x00\xd8\x00\xa7\x00 \x00\xd8\x00\xaa\x00\xd8\x00\xad\x00\xd8\x00\xb2\x00\xd9\x00\x86\x00'

In [35]:
# let's examine the columns we have
df.columns

Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'dir', 'rating_count', 'name', 'author', 'genres'], dtype='object')

Create the GroupBy table

In [144]:
authors = df.copy()
authors = authors[['rating','name','author']].groupby('author').agg({'rating' : np.mean,
                                                                    'name' : '|'.join})

In [145]:
authors.head()

,rating,name
author,,
A_A_Milne,4.365,Winnie-the-Pooh|The House at Pooh Corner|The H...
A_G_Howard,4.020,"Splintered (Splintered, #1)"
A_J_Cronin,4.220,The Keys of the Kingdom
A_J_Jacobs,3.750,The Year of Living Biblically
A_J_Salt,4.940,Nik Nassa & the Mark of Destiny


In [147]:
authors = authors.reset_index()
authors

,index,author,rating,name
0,0,A_A_Milne,4.365000,Winnie-the-Pooh|The House at Pooh Corner|The H...
1,1,A_G_Howard,4.020000,"Splintered (Splintered, #1)"
2,2,A_J_Cronin,4.220000,The Keys of the Kingdom
3,3,A_J_Jacobs,3.750000,The Year of Living Biblically
4,4,A_J_Salt,4.940000,Nik Nassa & the Mark of Destiny
...,...,...,...,...
2640,2640,_,3.988095,Ø¹Ø²Ø§Ø²ÙÙ|Ø«ÙØ§Ø«ÙØ© ØºØ±ÙØ§Ø·Ø©|ØªØ±Ø§Ø...
2641,2641,_gota_Krist_f,4.340000,"The Notebook, The Proof, The Third Lie"
2642,2642,_mile_Zola,3.990000,"Germinal (Les Rougon-Macquart, #13)|L'Assommoi..."
2643,2643,_ric_Emmanuel_Schmitt,4.160000,Oscar et la dame rose


In [149]:
# split the column string and make a list of string book names
authors['name'] = authors.name.str.split('|')
authors.head()

,index,author,rating,name
0,0,A_A_Milne,4.365,"[Winnie-the-Pooh, The House at Pooh Corner, Th..."
1,1,A_G_Howard,4.020,"[Splintered (Splintered, #1)]"
2,2,A_J_Cronin,4.220,[The Keys of the Kingdom]
3,3,A_J_Jacobs,3.750,[The Year of Living Biblically]
4,4,A_J_Salt,4.940,[Nik Nassa & the Mark of Destiny]


In [39]:
# count the books - create new column
len(authors.name[0])

6

In [157]:
authors['# of books'] = authors['name'].str.len()

# most prolific authors
np.max(authors['# of books'])

NameError: name 'location' is not defined

In [158]:
authors['num_books'] = authors['name'].str.len()
authors

,index,author,rating,name,# of books,num_books
0,0,A_A_Milne,4.365000,"[Winnie-the-Pooh, The House at Pooh Corner, Th...",6,6
1,1,A_G_Howard,4.020000,"[Splintered (Splintered, #1)]",1,1
2,2,A_J_Cronin,4.220000,[The Keys of the Kingdom],1,1
3,3,A_J_Jacobs,3.750000,[The Year of Living Biblically],1,1
4,4,A_J_Salt,4.940000,[Nik Nassa & the Mark of Destiny],1,1
...,...,...,...,...,...,...
2640,2640,_,3.988095,"[Ø¹Ø²Ø§Ø²ÙÙ, Ø«ÙØ§Ø«ÙØ© ØºØ±ÙØ§Ø·Ø©, ØªØ±...",42,42
2641,2641,_gota_Krist_f,4.340000,"[The Notebook, The Proof, The Third Lie]",1,1
2642,2642,_mile_Zola,3.990000,"[Germinal (Les Rougon-Macquart, #13), L'Assomm...",4,4
2643,2643,_ric_Emmanuel_Schmitt,4.160000,[Oscar et la dame rose],1,1


In [161]:
# sort for more prolific
authors.sort_values(by='num_books', ascending=False).iloc[0]
authors.sort_values(by='num_books', ascending=False).iloc[0:10]

,index,author,rating,name,# of books,num_books
2349,2349,Stephen_King,3.918750,"[The Stand, The Shining (The Shining #1), It, ...",56,56
2640,2640,_,3.988095,"[Ø¹Ø²Ø§Ø²ÙÙ, Ø«ÙØ§Ø«ÙØ© ØºØ±ÙØ§Ø·Ø©, ØªØ±...",42,42
2429,2429,Terry_Pratchett,4.103415,"[Good Omens, Night Watch (Discworld, #29), Sma...",41,41
1745,1745,Meg_Cabot,3.890000,"[The Princess Diaries (The Princess Diaries, #...",33,33
1494,1494,Laurell_K_Hamilton,3.954815,"[Guilty Pleasures (Anita Blake, Vampire Hunter...",27,27
704,704,Erin_Hunter,4.325200,"[Into the Wild (Warriors, #1), Fire and Ice (W...",25,25
1043,1043,James_Patterson,3.949600,"[The Angel Experiment (Maximum Ride, #1), Scho...",25,25
1866,1866,Neil_Gaiman,4.335200,"[American Gods (American Gods, #1), Neverwhere...",25,25
2103,2103,Richelle_Mead,4.292727,"[Vampire Academy (Vampire Academy, #1), Shadow...",22,22
2293,2293,Sherrilyn_Kenyon,4.269545,"[Acheron (Dark-Hunter, #8), Fantasy Lover (Dar...",22,22


#### Winner is Stephen King with 56 books! OMG!!!

Perhaps you want more detailed info...

In [162]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

rating                                                        rating_count                                                                             review_count                                                                      year                                                                   
                       count      mean       std   min     25%    50%     75%   max        count          mean           std      min       25%      50%       75%       max        count         mean          std     min      25%     50%      75%     max count         mean         std     min      25%     50%      75%     max
author                                                                                                                                                                                                                                                                                                                                
A_A_Milne                6.0  4.365000  0.078677  4.25  4.3275  4.365  4.4025  4.48          6.0  47842.333333  57135.314010    544.0  15559.50  30547.0  50401.25  157833.0          6.0   543.000000   682.473150     1.0   219.50   309.5   499.25  1886.0   6.0  1944.166667   29.294482  1926.0  1926.25  1927.5  1952.75  1997.0
A_G_Howard               1.0  4.020000       NaN  4.02  4.0200  4.020  4.0200  4.02          1.0  17073.000000           NaN  17073.0  17073.00  17073.0  17073.00   17073.0          1.0  3194.000000          NaN  3194.0  3194.00  3194.0  3194.00  3194.0   1.0  2013.000000         NaN  2013.0  2013.00  2013.0  2013.00  2013.0
A_J_Cronin               1.0  4.220000       NaN  4.22  4.2200  4.220  4.2200  4.22          1.0   1015.000000           NaN   1015.0   1015.00   1015.0   1015.00    1015.0          1.0   109.000000          NaN   109.0   109.00   109.0   109.00   109.0   1.0  1941.000000         NaN  1941.0  1941.00  1941.0  1941.00  1941.0
A_J_Jacobs               1.0  3.750000       NaN  3.75  3.7500  3.750  3.7500  3.75          1.0  39489.000000           NaN  39489.0  39489.00  39489.0  39489.00   39489.0          1.0  4371.000000          NaN  4371.0  4371.00  4371.0  4371.00  4371.0   1.0  2007.000000         NaN  2007.0  2007.00  2007.0  2007.00  2007.0
A_J_Salt                 1.0  4.940000       NaN  4.94  4.9400  4.940  4.9400  4.94          1.0     16.000000           NaN     16.0     16.00     16.0     16.00      16.0          1.0     6.000000          NaN     6.0     6.00     6.0     6.00     6.0   1.0  2014.000000         NaN  2014.0  2014.00  2014.0  2014.00  2014.0
...                      ...       ...       ...   ...     ...    ...     ...   ...          ...           ...           ...      ...       ...      ...       ...       ...          ...          ...          ...     ...      ...     ...      ...     ...   ...          ...         ...     ...      ...     ...      ...     ...
_                       42.0  3.988095  0.301211  3.15  3.8000  4.075  4.2075  4.47         42.0  11574.642857   9368.218450     10.0   4724.00   8709.5  15572.75   39302.0         42.0  1437.214286  1216.421371     1.0   534.75  1020.5  1745.75  4785.0  42.0  1998.904762   18.721071  1941.0  1994.75  2007.0  2010.75  2014.0
_gota_Krist_f            1.0  4.340000       NaN  4.34  4.3400  4.340  4.3400  4.34          1.0   2628.000000           NaN   2628.0   2628.00   2628.0   2628.00    2628.0          1.0   243.000000          NaN   243.0   243.00   243.0   243.00   243.0   1.0  1986.000000         NaN  1986.0  1986.00  1986.0  1986.00  1986.0
_mile_Zola               4.0  3.990000  0.100995  3.84  3.9750  4.035  4.0500  4.05          4.0   8358.250000   4553.226942   3594.0   5181.75   8095.0  11271.50   13649.0          4.0   283.750000   171.069917   125.0   189.50   243.5   337.75   523.0   4.0  1652.500000  456.345264   968.0  1649.75  1878.5  1881.25  1885.0
_ric_Emmanuel_Schmitt    1.0  4.160000       NaN  4.16  4.1600  4.160  4.1600  4.16          1.0  

You can also access a `groupby` dictionary style.

In [163]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
ratingdict

{'A_A_Milne': (4.364999999999999, 0.0786765530510839),
 'A_G_Howard': (4.02, nan),
 'A_J_Cronin': (4.22, nan),
 'A_J_Jacobs': (3.75, nan),
 'A_J_Salt': (4.94, nan),
 'A_Meredith_Walters': (4.15, 0.16970562748477094),
 'A_N_Roquelaure': (3.45, 0.2262741699796951),
 'A_S_Byatt': (3.86, nan),
 'A_S_King': (3.93, nan),
 'A_id_al_Qarni': (4.050000000000001, 0.19798989873223347),
 'Abbi_Glines': (4.179285714285713, 0.12916681436834276),
 'Abdul_Rahman_Munif': (4.03, nan),
 'Abigail_Gibbs': (3.82, nan),
 'Abigail_Roux': (4.47, 0.22876479915697992),
 'Abigail_Thomas': (3.68, nan),
 'Abolqasem_Ferdowsi': (4.52, nan),
 'Abraham_Verghese': (4.26, nan),
 'Abul_Hasan_Ali_Nadwi': (4.15, nan),
 'Adam_Hochschild': (4.14, nan),
 'Adam_Johnson': (4.03, nan),
 'Adam_Levin': (4.04, nan),
 'Adam_Rex': (4.14, nan),
 'Adam_Smith': (3.82, nan),
 'Addison_Moore': (3.78, nan),
 'Adeline_Yen_Mah': (4.01, nan),
 'Adolf_Hitler': (2.97, nan),
 'Adolfo_Bioy_Casares': (4.06, nan),
 'Aeschylus': (3.96, nan),
 'Aesop':

<div class="exercise"><b>Exercise</b></div>

Lets get the best-rated book(s) for every year in our dataframe.

In [170]:
# Group by year
dfgb_year = df.groupby('year')
dfgb_year.describe()



rating                                                        review_count                                                                      rating_count                                                                                   
       count      mean       std   min     25%    50%     75%   max        count         mean          std     min     25%     50%      75%       max        count           mean            std       min        25%       50%        75%        max
year                                                                                                                                                                                                                                                 
-1500    1.0  3.600000       NaN  3.60  3.6000  3.600  3.6000  3.60          1.0  1644.000000          NaN  1644.0  1644.0  1644.0  1644.00    1644.0          1.0   42026.000000            NaN   42026.0   42026.00   42026.0   42026.00    42026.0
-800     2.0  3.845000  0.233345  3.68  3.7625  3.845  3.9275  4.01          2.0  3075.000000  3832.518754   365.0  1720.0  3075.0  4430.00    5785.0          2.0  297685.500000  371319.448471   35123.0  166404.25  297685.5  428966.75   560248.0
-560     1.0  4.030000       NaN  4.03  4.0300  4.030  4.0300  4.03          1.0   890.000000          NaN   890.0   890.0   890.0   890.00     890.0          1.0   71259.000000            NaN   71259.0   71259.00   71259.0   71259.00    71259.0
-512     1.0  3.920000       NaN  3.92  3.9200  3.920  3.9200  3.92          1.0  3559.000000          NaN  3559.0  3559.0  3559.0  3559.00    3559.0          1.0  114619.000000            NaN  114619.0  114619.00  114619.0  114619.00   114619.0
-500     1.0  4.060000       NaN  4.06  4.0600  4.060  4.0600  4.06          1.0  1087.000000          NaN  1087.0  1087.0  1087.0  1087.00    1087.0          1.0   31634.000000            NaN   31634.0   31634.00   31634.0   31634.00    31634.0
...      ...       ...       ...   ...     ...    ...     ...   ...          ...          ...          ...     ...     ...     ...      ...       ...          ...            ...            ...       ...        ...       ...        ...        ...
 2010  317.0  4.046341  0.241775  3.08  3.8900  4.030  4.2000  4.93        317.0  2695.466877  6013.533198     0.0   583.0  1205.0  2800.00   83643.0        317.0   34873.738170   85223.744788      12.0    8257.00   17762.0   33320.00  1316704.0
 2011  374.0  4.067166  0.269598  2.00  3.9100  4.085  4.2475  5.00        374.0  3165.788770  6590.347074     0.0   567.0  1347.5  3392.75   82098.0        374.0   36757.652406   88235.558577       9.0    6763.00   15761.5   36190.75  1127983.0
 2012  355.0  4.154169  0.293960  3.15  3.9600  4.160  4.3400  5.00        355.0  3374.735211  8616.898114     0.0   320.0  1423.0  3400.00  112279.0        355.0   34245.698592   85374.157344       6.0    3132.50   15955.0   35518.50  1150626.0
 2013  276.0  4.200978  0.287850  2.90  4.0400  4.230  4.3725  4.93        276.0  2477.271739  4610.929059     1.0    78.5  1346.5  2616.50   47308.0        276.0   20527.981884   35280.487254       7.0     251.00   11233.0   24611.50   358301.0
 2014   87.0  4.349655  0.349613  3.31  4.1400  4.370  4.5900  5.00         87.0  1532.436782  2307.469916     1.0    22.0   273.0  2396.50   11011.0         87.0    9262.678161   14208.785183       5.0      51.00     792.0   13499.00    69924.0

[293 rows x 24 columns]

In [178]:
bestbooks_by_year =[]
for year, subset in dfgb_year:
    bestbook = subset[subset.rating==subset.rating.max()]
    temp = [year,bestbook.name.values,bestbook.rating.values]
    bestbooks_by_year.append(temp)
bestbooks_by_year

[[-1500, array(['The Epic of Gilgamesh'], dtype=object), array([3.6])],
 [-800, array(['The Iliad/The Odyssey'], dtype=object), array([4.01])],
 [-560, array(["Aesop's Fables"], dtype=object), array([4.03])],
 [-512, array(['The Art of War'], dtype=object), array([3.92])],
 [-500, array(['The Bhagavad Gita'], dtype=object), array([4.06])],
 [-458, array(['The Oresteia'], dtype=object), array([3.96])],
 [-442, array(['Antigone'], dtype=object), array([3.52])],
 [-440, array(['The Histories'], dtype=object), array([3.94])],
 [-431, array(['Medea'], dtype=object), array([3.8])],
 [-429, array(['Oedipus Rex'], dtype=object), array([3.64])],
 [-411,
  array(['The History of the Peloponnesian War'], dtype=object),
  array([3.84])],
 [-400, array(['Complete Works'], dtype=object), array([4.3])],
 [-390, array(['Apology'], dtype=object), array([4.11])],
 [-380, array(['The Republic'], dtype=object), array([3.85])],
 [-370, array(['The Symposium'], dtype=object), array([3.99])],
 [-360, array([

In [173]:
df.sort_values(by='rating',ascending=False).iloc[0];
#authors.sort_values(by='num_books', ascending=False).iloc[0]

In [179]:
#Using .groupby, we can divide the dataframe into subsets by the values of 'year'.
#We can then iterate over these subsets
# your code here
for year, subset in df.groupby('year'):
    #Find the best book of the year

    bestbook = subset[subset.rating == subset.rating.max()]
    if bestbook.shape[0] > 1:
        print(year, bestbook.name.values, bestbook.rating.values)
    else:
        print(year, bestbook.name.values[0], bestbook.rating.values[0])

-1500 The Epic of Gilgamesh 3.6
-800 The Iliad/The Odyssey 4.01
-560 Aesop's Fables 4.03
-512 The Art of War 3.92
-500 The Bhagavad Gita 4.06
-458 The Oresteia 3.96
-442 Antigone 3.52
-440 The Histories 3.94
-431 Medea 3.8
-429 Oedipus Rex 3.64
-411 The History of the Peloponnesian War 3.84
-400 Complete Works 4.3
-390 Apology 4.11
-380 The Republic 3.85
-370 The Symposium 3.99
-360 Five Dialogues 4.07
-350 The Nicomachean Ethics 3.88
-29 The Aeneid 3.77
4 The Seven Spiritual Laws of Success 4.04
8 Metamorphoses 4.01
13 The Day Jesus Rode Into Croydon 3.85
100 The New Oxford Annotated Bible, New Revised Standard Version 4.3
121 The Twelve Caesars 4.02
130 Holy Bible 4.45
158 The Golden Ass 3.88
174 Meditations 4.16
200 The Garden of Eden 3.75
201 Fe Fi FOE Comes 4.37
213 NaÃ¯ve. Super 3.9
397 Confessions 3.82
632 The Quran 4.37
800 Beowulf 3.38
900 The Arabian Nights 4.04
968 La BÃªte humaine (Les Rougon-Macquart, #17) 4.02
1002 The Pillow Book 4.04
1008 The Tale of Genji 3.68
1010 Sha